Variational AutoEncoder (VAE) with CelebA
=========================================
---
Formation Introduction au Deep Learning  (FIDLE) - S. Arias, E. Maldonado, JL. Parouty - CNRS/SARI/DEVLOG - 2020  

## Episode 1 - Train a model

 - Defining a VAE model
 - Build the model
 - Train it
 - Follow the learning process with Tensorboard


## Step 1 - Setup environment
### 1.1 - Python stuff

In [11]:
import tensorflow as tf
import numpy as np
import os,sys
from importlib import reload

import modules.vae
import modules.data_generator

reload(modules.data_generator)
reload(modules.vae)

from modules.vae  import VariationalAutoencoder
from modules.data_generator import DataGenerator

sys.path.append('..')
import fidle.pwk as ooo
reload(ooo)

ooo.init()

VariationalAutoencoder.about()
DataGenerator.about()


FIDLE 2020 - Practical Work Module
Version              : 0.2.8
Run time             : Thursday 13 February 2020, 00:18:26
TensorFlow version   : 2.0.0
Keras version        : 2.2.4-tf

FIDLE 2020 - Variational AutoEncoder (VAE)
TensorFlow version   : 2.0.0
VAE version          : 1.27

FIDLE 2020 - DataGenerator
Version              : 0.4


### 1.2 - The good place

In [2]:
place, dataset_dir = ooo.good_place( { 'GRICAD' : f'{os.getenv("SCRATCH_DIR","")}/PROJECTS/pr-fidle/datasets/celeba',
                                       'IDRIS'  : f'{os.getenv("WORK","")}/datasets/celeba'    } )

# ---- train/test datasets

train_dir    = f'{dataset_dir}/clusters.train'
test_dir     = f'{dataset_dir}/clusters.test'

Well, we should be at GRICAD !
We are going to use: /bettik/PROJECTS/pr-fidle/datasets/celeba


## Step 2 - DataGenerator and validation data
Ok, everything's perfect, now let's instantiate our generator for the entire dataset.

In [3]:
data_gen = DataGenerator(train_dir, 32, k_size=1)
x_test   = np.load(f'{test_dir}/images-000.npy')

print(f'Data generator : {len(data_gen)} batchs of {data_gen.batch_size} images, or {data_gen.dataset_size} images')
print(f'x_test : {len(x_test)} images')

Data generator : 6250 batchs of 32 images, or 200000 images
x_test : 2599 images


## Step 3 - Get VAE model

In [9]:
tag = 'CelebA.004'

input_shape = (128, 128, 3)
z_dim       = 200
verbose     = 1

encoder= [ {'type':'Conv2D',          'filters':32, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Dropout',         'rate':0.25},
           {'type':'Conv2D',          'filters':64, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Dropout',         'rate':0.25},
           {'type':'Conv2D',          'filters':64, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Dropout',         'rate':0.25},
           {'type':'Conv2D',          'filters':64, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Dropout',         'rate':0.25},
         ]

decoder= [ {'type':'Conv2DTranspose', 'filters':64, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Dropout',         'rate':0.25},
           {'type':'Conv2DTranspose', 'filters':64, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Dropout',         'rate':0.25},
           {'type':'Conv2DTranspose', 'filters':32, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Dropout',         'rate':0.25},
           {'type':'Conv2DTranspose', 'filters':3,  'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'sigmoid'}
         ]

vae = modules.vae.VariationalAutoencoder(input_shape    = input_shape, 
                                         encoder_layers = encoder, 
                                         decoder_layers = decoder,
                                         z_dim          = z_dim, 
                                         verbose        = verbose,
                                         run_tag        = tag)
vae.save(model=None)

Model initialized.
Outputs will be in  : ./run/CelebA.004

 ---------- Encoder -------------------------------------------------- 

Model: "model_21"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, 240, 160, 3) 0                                            
__________________________________________________________________________________________________
conv2d_20 (Conv2D)              (None, 120, 80, 32)  896         encoder_input[0][0]              
__________________________________________________________________________________________________
dropout_35 (Dropout)            (None, 120, 80, 32)  0           conv2d_20[0][0]                  
__________________________________________________________________________________________________
conv2d_21 (Conv2D)              (None, 60, 40, 64)   18496

## Step 4 - Compile it

In [12]:
optimizer = tf.keras.optimizers.Adam(1e-4)
# optimizer     = 'adam'
r_loss_factor = 10000

vae.compile(optimizer, r_loss_factor)

Compiled.


## Step 5 - Train
For 10 epochs, adam optimizer :  
- Run time at IDRIS : 1299.77 sec. - 0:21:39
- Run time at GRICAD : 2092.77 sec. - 0:34:52

In [10]:
epochs            = 10
initial_epoch     = 0

In [11]:
vae.train(data_generator    = data_gen,
          x_test            = x_test,
          epochs            = epochs,
          initial_epoch     = initial_epoch
         )

Epoch 1/10
6250/6250 [==============================] - 245s 39ms/step - loss: 270.7260 - vae_r_loss: 218.7500 - vae_kl_loss: 51.9758 - val_loss: 218.8331 - val_vae_r_loss: 166.5691 - val_vae_kl_loss: 52.0549
Epoch 2/10
6250/6250 [==============================] - 197s 32ms/step - loss: 225.6190 - vae_r_loss: 170.7332 - vae_kl_loss: 54.8861 - val_loss: 207.9924 - val_vae_r_loss: 154.9203 - val_vae_kl_loss: 52.8327
Epoch 3/10
6250/6250 [==============================] - 213s 34ms/step - loss: 221.2871 - vae_r_loss: 166.0321 - vae_kl_loss: 55.2546 - val_loss: 205.8002 - val_vae_r_loss: 151.2378 - val_vae_kl_loss: 54.3242
Epoch 4/10
6250/6250 [==============================] - 209s 33ms/step - loss: 219.1529 - vae_r_loss: 163.8348 - vae_kl_loss: 55.3184 - val_loss: 207.8092 - val_vae_r_loss: 156.0968 - val_vae_kl_loss: 51.4896
Epoch 5/10
6250/6250 [==============================] - 204s 33ms/step - loss: 217.8949 - vae_r_loss: 162.5502 - vae_kl_loss: 55.3446 - val_loss: 201.9843 - val_vae